In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib

from glob import glob

import requests # 카카오 맵 API 불러오기 위해

import csv # 마포구 데이터 csv 모듈로 리스트로 먼저 받아온 후 오류 수정, 데이터프레임화 하기 위해

## 강남구 쓰레기통

In [2]:
gangnam_cbb = pd.read_csv('data/쓰레기통/서울특별시 강남구_담배꽁초 전용 쓰레기통_20220808.csv', encoding='cp949')
gangnam_cbb.head()

,동명,설치주소,위 치
0,신사동,압구정로14길 6(신사동 548-4),코다차야 남동쪽(가게 앞)
1,신사동,강남대로152길 14(신사동 513-2),FLASH BAR 동쪽(전신주)
2,신사동,강남대로152길 29(신사동 515-12),인사동촌 남쪽(가게 앞)
3,신사동,강남대로152길 34(신사동 514-8),고향역(전신주)
4,논현1동,학동로2길 11 (취영루) 맞은편,영풍빌딩 주차장 옆


In [3]:
# 23, 24번째 값이 중복
gangnam_cbb[22:30]

,동명,설치주소,위 치
22,삼성1동,테헤란로81길 46,가게앞 모퉁이
23,삼성1동,테헤란로83길 12,제일주차장앞 보도상
24,삼성1동,테헤란로83길 12,제일주차장앞 보도상
25,삼성1동,삼성로92길 28,도로 모퉁이
26,삼성1동,삼성로96길 15,오징어세상가게 옆골목도로변
27,삼성1동,삼성로104길 22,주소지앞 모퉁이 보도상
28,삼성1동,봉은사로82길 32,주소지 앞 보도상
29,삼성1동,삼성로104길 17,주소지 앞 보도상


In [4]:
gangnam_cbb.shape

(45, 3)

In [5]:
gangnam_cbb = gangnam_cbb.drop_duplicates(ignore_index=True)

In [6]:
gangnam_cbb.shape

(44, 3)

In [7]:
# 설치주소 + (부연 설명) 있는 것을 제거하기 위함
gangnam_cbb['설치주소'] = gangnam_cbb['설치주소'].apply(lambda x: x.split('(')[0])

In [8]:
# 설치 주소 내 맞은편, 앞 등과 같은 단어 제거
a = gangnam_cbb['설치주소'].str.split(' ')
a.head()

0      [압구정로14길, 6]
1    [강남대로152길, 14]
2    [강남대로152길, 29]
3    [강남대로152길, 34]
4     [학동로2길, 11, ]
Name: 설치주소, dtype: object

In [9]:
for i in range(len(a)):
    if a[i][-1] == '맞은편':
        a[i] = a[i][:-1]
    elif a[i][-1] == '앞':
        a[i] = a[i][:-1]

In [10]:
for i in range(len(a)):
    if len(a[i]) == 1:
        a[i] = '서울특별시 강남구 ' + a[i][0]
    else:
        a[i] = '서울특별시 강남구 ' + a[i][0] + ' ' + a[i][1]

In [11]:
a

0       서울특별시 강남구 압구정로14길 6
1     서울특별시 강남구 강남대로152길 14
2     서울특별시 강남구 강남대로152길 29
3     서울특별시 강남구 강남대로152길 34
4        서울특별시 강남구 학동로2길 11
5        서울특별시 강남구 학동로2길 19
6        서울특별시 강남구 학동로2길 47
7       서울특별시 강남구 봉은사로1길 35
8       서울특별시 강남구 봉은사로1길 32
9       서울특별시 강남구 봉은사로1길 16
10      서울특별시 강남구 봉은사로1길 16
11       서울특별시 강남구 학동로2길 47
12       서울특별시 강남구 학동로2길 13
13       서울특별시 강남구 논현동 98-6
14      서울특별시 강남구 논현동 97-20
15      서울특별시 강남구 논현동 97-18
16      서울특별시 강남구 논현동 97-17
17      서울특별시 강남구 논현동 98-12
18      서울특별시 강남구 논현동 98-12
19     서울특별시 강남구 논현동 105-12
20      서울특별시 강남구 압구정로36길 8
21     서울특별시 강남구 삼성로100길 24
22     서울특별시 강남구 테헤란로81길 46
23     서울특별시 강남구 테헤란로83길 12
24      서울특별시 강남구 삼성로92길 28
25      서울특별시 강남구 삼성로96길 15
26     서울특별시 강남구 삼성로104길 22
27     서울특별시 강남구 봉은사로82길 32
28     서울특별시 강남구 삼성로104길 17
29     서울특별시 강남구 삼성로104길 28
30       서울특별시 강남구  테헤란로64길
31       서울특별시 강남구  테헤란로64길
32        서울특별시 강남구  선릉로86길
33        서울특별시 강남구  삼성로85길
34        서울특별시 강남구  선릉로90길
35        서울특별시 강남구 

In [12]:
gangnam_cbb['설치주소'] = a

In [13]:
gangnam_cbb.head()

,동명,설치주소,위 치
0,신사동,서울특별시 강남구 압구정로14길 6,코다차야 남동쪽(가게 앞)
1,신사동,서울특별시 강남구 강남대로152길 14,FLASH BAR 동쪽(전신주)
2,신사동,서울특별시 강남구 강남대로152길 29,인사동촌 남쪽(가게 앞)
3,신사동,서울특별시 강남구 강남대로152길 34,고향역(전신주)
4,논현1동,서울특별시 강남구 학동로2길 11,영풍빌딩 주차장 옆


In [14]:
# address = '서울특별시 강남구 압구정로14길 6'
# url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
# headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
# results = requests.get(url, headers=headers).json()['documents']
# results

In [15]:
# address = '서울특별시 강남구 삼성로92길 28'
# url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
# headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
# results = requests.get(url, headers=headers).json()['documents']
# results

In [16]:
# 위경도 매칭 함수
def GetLatLng(address):
    
    url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address) #요청할 url 주소
    headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'} #REST API 키(유효한 키)
    results = requests.get(url,
                          headers=headers).json()['documents'] #카카오 API 요청
    
    result = results[0]
    name = result['address_name']
    x = result['x'] # Lng, 경도
    y = result['y'] # Lat, 위도
    data = [name, x, y, address]
    return data

In [17]:
GetLatLng('서울특별시 강남구 압구정로14길 6')

['서울 강남구 압구정로14길 6',
 '127.022980226111',
 '37.523858489259',
 '서울특별시 강남구 압구정로14길 6']

In [18]:
# '서울특별시 강남구 삼성로92길 28' 이 위경도 매칭 오류가 나서 27과 29의 위경도가 얼마나 다른가 확인
GetLatLng('서울특별시 강남구 삼성로92길 27')

['서울 강남구 삼성로92길 27',
 '127.058532226672',
 '37.5083786932357',
 '서울특별시 강남구 삼성로92길 27']

In [19]:
GetLatLng('서울특별시 강남구 삼성로92길 29')

['서울 강남구 삼성로92길 29',
 '127.058778402547',
 '37.5083178432764',
 '서울특별시 강남구 삼성로92길 29']

In [20]:
gangnam_cbb.loc[gangnam_cbb['설치주소']=='서울특별시 강남구 삼성로92길 28', '설치주소'] = '서울특별시 강남구 삼성로92길 27'

In [21]:
gangnamlist = []
for i in list(gangnam_cbb['설치주소']):
    gangnamlist.append(GetLatLng(i))
    print(GetLatLng(i))

['서울 강남구 압구정로14길 6', '127.022980226111', '37.523858489259', '서울특별시 강남구 압구정로14길 6']
['서울 강남구 강남대로152길 14', '127.020699344217', '37.5171506735807', '서울특별시 강남구 강남대로152길 14']
['서울 강남구 강남대로152길 29', '127.021697932358', '37.5179477021889', '서울특별시 강남구 강남대로152길 29']
['서울 강남구 강남대로152길 34', '127.02210634185', '37.5177615692586', '서울특별시 강남구 강남대로152길 34']
['서울 강남구 학동로2길 11', '127.02273341768', '37.5105392729193', '서울특별시 강남구 학동로2길 11']
['서울 강남구 학동로2길 19', '127.022960047655', '37.5100607055782', '서울특별시 강남구 학동로2길 19']
['서울 강남구 학동로2길 47', '127.023672833289', '37.5086256248304', '서울특별시 강남구 학동로2길 47']
['서울 강남구 봉은사로1길 35', '127.024049967581', '37.5070191469956', '서울특별시 강남구 봉은사로1길 35']
['서울 강남구 봉은사로1길 32', '127.024515199807', '37.5068294798978', '서울특별시 강남구 봉은사로1길 32']
['서울 강남구 봉은사로1길 16', '127.024941721878', '37.5058765776821', '서울특별시 강남구 봉은사로1길 16']
['서울 강남구 봉은사로1길 16', '127.024941721878', '37.5058765776821', '서울특별시 강남구 봉은사로1길 16']
['서울 강남구 학동로2길 47', '127.023672833289', '37.5086256248304', '서울특별시 강남구 학동

In [22]:
# '서울특별시 강남구 삼성로92길 28'은 제외하고 생성
# 23/05/25 업데이트. '서울특별시 강남구 삼성로92길 27'로 변경 후 리스트 추가
gangnamlist

[['서울 강남구 압구정로14길 6',
  '127.022980226111',
  '37.523858489259',
  '서울특별시 강남구 압구정로14길 6'],
 ['서울 강남구 강남대로152길 14',
  '127.020699344217',
  '37.5171506735807',
  '서울특별시 강남구 강남대로152길 14'],
 ['서울 강남구 강남대로152길 29',
  '127.021697932358',
  '37.5179477021889',
  '서울특별시 강남구 강남대로152길 29'],
 ['서울 강남구 강남대로152길 34',
  '127.02210634185',
  '37.5177615692586',
  '서울특별시 강남구 강남대로152길 34'],
 ['서울 강남구 학동로2길 11',
  '127.02273341768',
  '37.5105392729193',
  '서울특별시 강남구 학동로2길 11'],
 ['서울 강남구 학동로2길 19',
  '127.022960047655',
  '37.5100607055782',
  '서울특별시 강남구 학동로2길 19'],
 ['서울 강남구 학동로2길 47',
  '127.023672833289',
  '37.5086256248304',
  '서울특별시 강남구 학동로2길 47'],
 ['서울 강남구 봉은사로1길 35',
  '127.024049967581',
  '37.5070191469956',
  '서울특별시 강남구 봉은사로1길 35'],
 ['서울 강남구 봉은사로1길 32',
  '127.024515199807',
  '37.5068294798978',
  '서울특별시 강남구 봉은사로1길 32'],
 ['서울 강남구 봉은사로1길 16',
  '127.024941721878',
  '37.5058765776821',
  '서울특별시 강남구 봉은사로1길 16'],
 ['서울 강남구 봉은사로1길 16',
  '127.024941721878',
  '37.5058765776821',
  '서울특별시 강남

In [23]:
gangnam_latlng = pd.DataFrame(gangnamlist)
gangnam_latlng.columns = ['address_name', '경도', '위도', '설치주소']
gangnam_latlng.head()

,address_name,경도,위도,설치주소
0,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6
1,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14
2,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29
3,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34
4,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11


In [24]:
gangnam_m = pd.merge(gangnam_latlng, gangnam_cbb, on=['설치주소'], how='left')
gangnam_m.drop_duplicates
print(gangnam_m.shape)
gangnam_m.head()

(54, 6)


,address_name,경도,위도,설치주소,동명,위 치
0,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6,신사동,코다차야 남동쪽(가게 앞)
1,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14,신사동,FLASH BAR 동쪽(전신주)
2,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29,신사동,인사동촌 남쪽(가게 앞)
3,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34,신사동,고향역(전신주)
4,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11,논현1동,영풍빌딩 주차장 옆


In [25]:
gangnam_m['시군구명']='강남구'

In [26]:
gangnam_m = gangnam_m[['시군구명', '동명', 'address_name', '경도', '위도', '설치주소', '위  치']]
gangnam_m.head()

,시군구명,동명,address_name,경도,위도,설치주소,위 치
0,강남구,신사동,서울 강남구 압구정로14길 6,127.022980226111,37.523858489259,서울특별시 강남구 압구정로14길 6,코다차야 남동쪽(가게 앞)
1,강남구,신사동,서울 강남구 강남대로152길 14,127.020699344217,37.5171506735807,서울특별시 강남구 강남대로152길 14,FLASH BAR 동쪽(전신주)
2,강남구,신사동,서울 강남구 강남대로152길 29,127.021697932358,37.5179477021889,서울특별시 강남구 강남대로152길 29,인사동촌 남쪽(가게 앞)
3,강남구,신사동,서울 강남구 강남대로152길 34,127.02210634185,37.5177615692586,서울특별시 강남구 강남대로152길 34,고향역(전신주)
4,강남구,논현1동,서울 강남구 학동로2길 11,127.02273341768,37.5105392729193,서울특별시 강남구 학동로2길 11,영풍빌딩 주차장 옆


In [27]:
gangnam_m.to_csv('data/쓰레기통_위경도/강남구_담배꽁초쓰레기통_위경도.csv', index=False)

## 동작구 쓰레기통

In [28]:
dongjak_cbb = pd.read_csv('data/쓰레기통/서울특별시 동작구_담배꽁초 전용 쓰레기통 현황_20230427.csv', encoding='cp949')
dongjak_cbb.head()

,연번,설치장소(상호명),도로명주소,비고(관리연합회),데이터 기준일자
0,1,노량진생고기,서울특별시 동작구 만양로 108,상가연합회,2023-04-27
1,2,두꺼비식당,서울특별시 동작구 만양로 102-3,상가연합회,2023-04-27
2,3,명가왕족발보쌈,서울특별시 동작구 만양로 92,상가연합회,2023-04-27
3,4,짚신매운갈비,서울특별시 동작구 만양로 94,상가연합회,2023-04-27
4,5,노량해전,서울특별시 동작구 만양로 16길 4,상가연합회,2023-04-27


In [29]:
dongjaklist = []

In [30]:
for i in list(dongjak_cbb['도로명주소']):
    dongjaklist.append(GetLatLng(i))
    print(GetLatLng(i))

['서울 동작구 만양로 108', '126.944433457305', '37.5130680513174', '서울특별시 동작구 만양로 108']
['서울 동작구 만양로 102-3', '126.944736476626', '37.5126085913106', '서울특별시 동작구 만양로 102-3']
['서울 동작구 만양로 92', '126.944477404179', '37.5116122295125', '서울특별시 동작구 만양로 92']
['서울 동작구 만양로 94', '126.944447779056', '37.5117527722431', '서울특별시 동작구 만양로 94']
['서울 동작구 만양로16길 4', '126.944668473774', '37.5115693412681', '서울특별시 동작구 만양로 16길 4']
['서울 동작구 만양로16길 6', '126.94483086714', '37.5116078901084', '서울특별시 동작구 만양로 16길 6']
['서울 동작구 만양로16길 9', '126.944625104341', '37.5117893461774', '서울특별시 동작구 만양로 16길 9']
['서울 동작구 만양로14길 9', '126.94496936875', '37.5115282158397', '서울특별시 동작구 만양로 14길 9']
['서울 동작구 만양로 89-1', '126.944194837851', '37.5114935241946', '서울특별시 동작구 만양로 89-1']
['서울 동작구 만양로 85-1', '126.944325977865', '37.511122912915', '서울특별시 동작구 만양로 85-1']
['서울 동작구 만양로14길 62', '126.947576376456', '37.5124921246185', '서울특별시 동작구 만양로14길 62']
['서울 동작구 만양로14아길 11', '126.948129062487', '37.5121101626587', '서울특별시 동작구 만양로14아길 11']
['서울 동작구 만양로12가길 

In [31]:
dongjak_latlng = pd.DataFrame(dongjaklist)
dongjak_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
dongjak_latlng.head()

,address_name,경도,위도,도로명주소
0,서울 동작구 만양로 108,126.944433457305,37.5130680513174,서울특별시 동작구 만양로 108
1,서울 동작구 만양로 102-3,126.944736476626,37.5126085913106,서울특별시 동작구 만양로 102-3
2,서울 동작구 만양로 92,126.944477404179,37.5116122295125,서울특별시 동작구 만양로 92
3,서울 동작구 만양로 94,126.944447779056,37.5117527722431,서울특별시 동작구 만양로 94
4,서울 동작구 만양로16길 4,126.944668473774,37.5115693412681,서울특별시 동작구 만양로 16길 4


In [32]:
dongjak_m = pd.merge(dongjak_latlng, dongjak_cbb, on='도로명주소', how='left')
dongjak_m.drop_duplicates()
print(dongjak_m.shape)
dongjak_m.head()

(17, 8)


,address_name,경도,위도,도로명주소,연번,설치장소(상호명),비고(관리연합회),데이터 기준일자
0,서울 동작구 만양로 108,126.944433457305,37.5130680513174,서울특별시 동작구 만양로 108,1,노량진생고기,상가연합회,2023-04-27
1,서울 동작구 만양로 102-3,126.944736476626,37.5126085913106,서울특별시 동작구 만양로 102-3,2,두꺼비식당,상가연합회,2023-04-27
2,서울 동작구 만양로 92,126.944477404179,37.5116122295125,서울특별시 동작구 만양로 92,3,명가왕족발보쌈,상가연합회,2023-04-27
3,서울 동작구 만양로 94,126.944447779056,37.5117527722431,서울특별시 동작구 만양로 94,4,짚신매운갈비,상가연합회,2023-04-27
4,서울 동작구 만양로16길 4,126.944668473774,37.5115693412681,서울특별시 동작구 만양로 16길 4,5,노량해전,상가연합회,2023-04-27


In [33]:
dongjak_m['시군구명']='동작구'

In [34]:
dongjak_m.columns

Index(['address_name', '경도', '위도', '도로명주소', '연번', '설치장소(상호명)', '비고(관리연합회)',
       '데이터 기준일자', '시군구명'],
      dtype='object')

In [35]:
dongjak_m = dongjak_m.drop(columns=['연번', '데이터 기준일자'])

In [36]:
dongjak_m = dongjak_m.rename(columns = {'도로명주소' : '설치주소'})

In [37]:
dongjak_m = dongjak_m[['시군구명', 'address_name', '경도', '위도', '설치주소', '설치장소(상호명)', '비고(관리연합회)']]
dongjak_m.head()

,시군구명,address_name,경도,위도,설치주소,설치장소(상호명),비고(관리연합회)
0,동작구,서울 동작구 만양로 108,126.944433457305,37.5130680513174,서울특별시 동작구 만양로 108,노량진생고기,상가연합회
1,동작구,서울 동작구 만양로 102-3,126.944736476626,37.5126085913106,서울특별시 동작구 만양로 102-3,두꺼비식당,상가연합회
2,동작구,서울 동작구 만양로 92,126.944477404179,37.5116122295125,서울특별시 동작구 만양로 92,명가왕족발보쌈,상가연합회
3,동작구,서울 동작구 만양로 94,126.944447779056,37.5117527722431,서울특별시 동작구 만양로 94,짚신매운갈비,상가연합회
4,동작구,서울 동작구 만양로16길 4,126.944668473774,37.5115693412681,서울특별시 동작구 만양로 16길 4,노량해전,상가연합회


In [38]:
dongjak_m.to_csv('data/쓰레기통_위경도/동작구_담배꽁초쓰레기통_위경도.csv', index=False)

## 마포구 쓰레기통

In [39]:
file_path = 'data\쓰레기통\서울특별시 마포구 용강동 상점가 담배꽁초수거함 설치 현황_20220805.csv'
f = open(file_path, 'rt')
reader = csv.reader(f)

csv_list = []
for l in reader:
    csv_list.append(l)
f.close()

df = pd.DataFrame(csv_list)
df.head()

,0,1,2,3,4,5,6,7
0,시도명,시군구명,상호명,도로명주소,수량,세부위치,데이터기준일자,
1,서울특별시,마포구,마포나루장터,서울특별시 마포구 광성로6길 66 (쌍마막회에서 1개 이동),1,,2022-08-05,
2,서울특별시,마포구,블랙시스(아이나비),서울특별시 마포구 대흥로 40,1,가게앞,2022-08-05,
3,서울특별시,마포구,성우서서갈비,서울특별시 마포구 마포대로1길 26,2,가게안/가게앞,2022-08-05,
4,서울특별시,마포구,고고네 닭한마리,서울특별시 마포구 마포대로1길 46,1,가게앞,2022-08-05,


In [40]:
mapo_cbb = pd.read_csv('data/쓰레기통/서울특별시 마포구 용강동 상점가 담배꽁초수거함 설치 현황_20220805.csv')
mapo_cbb.head()

,시도명,시군구명,상호명,도로명주소,수량,세부위치,데이터기준일자,Unnamed: 7
0,서울특별시,마포구,마포나루장터,서울특별시 마포구 광성로6길 66 (쌍마막회에서 1개 이동),1,NaN,2022-08-05,NaN
1,서울특별시,마포구,블랙시스(아이나비),서울특별시 마포구 대흥로 40,1,가게앞,2022-08-05,NaN
2,서울특별시,마포구,성우서서갈비,서울특별시 마포구 마포대로1길 26,2,가게안/가게앞,2022-08-05,NaN
3,서울특별시,마포구,고고네 닭한마리,서울특별시 마포구 마포대로1길 46,1,가게앞,2022-08-05,NaN
4,서울특별시,마포구,원마산아구찜,서울특별시 마포구 마포대로1길 46 우측상가,1,재활용수집,2022-08-05,NaN


In [41]:
mapo_cbb = mapo_cbb.drop(columns=['Unnamed: 7'])

In [42]:
mapo_cbb['도로명주소'] = mapo_cbb['도로명주소'].apply(lambda x:x.split('(')[0])
mapo_cbb.head()

,시도명,시군구명,상호명,도로명주소,수량,세부위치,데이터기준일자
0,서울특별시,마포구,마포나루장터,서울특별시 마포구 광성로6길 66,1,NaN,2022-08-05
1,서울특별시,마포구,블랙시스(아이나비),서울특별시 마포구 대흥로 40,1,가게앞,2022-08-05
2,서울특별시,마포구,성우서서갈비,서울특별시 마포구 마포대로1길 26,2,가게안/가게앞,2022-08-05
3,서울특별시,마포구,고고네 닭한마리,서울특별시 마포구 마포대로1길 46,1,가게앞,2022-08-05
4,서울특별시,마포구,원마산아구찜,서울특별시 마포구 마포대로1길 46 우측상가,1,재활용수집,2022-08-05


In [43]:
# 주소 변경 데이터 1
# 토정로25길 26인 것일까
# 토정로 261인 것일까...
# 육도라는 상호명이 토정로 261임
address = '서울특별시 마포구 토정로25길 261'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

[]

In [44]:
# 주소 변경 데이터 2
# 석양집의 주소는 '토정로 317'
# 설치 세부 위치인 띠녹지 앞은 '토정로 35길 12' - 하지만 해당 주소로는 위경도가 안나옴
address = '서울특별시 마포구 토정로35길 12'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

[]

In [45]:
mapo_cbb.loc[mapo_cbb['도로명주소']=='서울특별시 마포구 토정로25길 261', '도로명주소'] = '서울특별시 마포구 토정로 261'

In [46]:
# 외 1건이라면, 수량에 2개를 설정해야 하는지 고민
mapo_cbb.loc[mapo_cbb['도로명주소']=='서울특별시 마포구 토정로35길 12외 1건', '도로명주소'] = '서울특별시 마포구 토정로 317'

In [47]:
mapo_cbb['도로명주소'][43:]

43                     서울특별시 마포구 토정로 261
44                   서울특별시 마포구 토정로31길 23
45        서울특별시 마포구 토정로31길 23 마포리버웰상가 1층
46    서울특별시 마포구 토정로31길 23 마포리버웰상가 주차장 창고
47                   서울특별시 마포구 토정로31길 35
48                    서울특별시 마포구 토정로33길 5
49                   서울특별시 마포구 토정로35길 11
50                     서울특별시 마포구 토정로 317
51                   서울특별시 마포구 토정로35길 26
52                   서울특별시 마포구 토정로35길 26
53                  서울특별시 마포구 토정로37가길 9 
54                   서울특별시 마포구 토정로37길 13
55                   서울특별시 마포구 토정로37길 17
56                   서울특별시 마포구 토정로37길 19
Name: 도로명주소, dtype: object

In [48]:
mapolist=[]
for i in list(mapo_cbb['도로명주소']):
    mapolist.append(GetLatLng(i))
    print(GetLatLng(i))

['서울 마포구 광성로6길 66', '126.940168393772', '37.547265277456', '서울특별시 마포구 광성로6길 66 ']
['서울 마포구 대흥로 40', '126.938931078126', '37.5442301695085', '서울특별시 마포구 대흥로 40']
['서울 마포구 마포대로1길 26', '126.942323015666', '37.540313878466', '서울특별시 마포구 마포대로1길 26']
['서울 마포구 마포대로1길 46', '126.940477382241', '37.5416404055047', '서울특별시 마포구 마포대로1길 46 ']
['서울 마포구 마포대로1길 46', '126.940477382241', '37.5416404055047', '서울특별시 마포구 마포대로1길 46 우측상가']
['서울 마포구 마포대로 25', '126.94483360828', '37.5393787588818', '서울특별시 마포구 마포대로 25']
['서울 마포구 토정로 252', '126.938303206822', '37.5432283824035', '서울특별시 마포구 토정로 252']
['서울 마포구 토정로 253', '126.93869987729', '37.5435664637455', '서울특별시 마포구 토정로 253']
['서울 마포구 토정로 255', '126.93886948072', '37.5434480702547', '서울특별시 마포구 토정로 255']
['서울 마포구 토정로 257', '126.939074320928', '37.5432730218428', '서울특별시 마포구 토정로 257']
['서울 마포구 토정로 261', '126.939396951664', '37.5431010969782', '서울특별시 마포구 토정로 261']
['서울 마포구 토정로 262', '126.939080446148', '37.5428375719887', '서울특별시 마포구 토정로 262']
['서울 마포구 토정로 272', '126.93

In [49]:
mapo_latlng = pd.DataFrame(mapolist)
mapo_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
mapo_latlng.head()

,address_name,경도,위도,도로명주소
0,서울 마포구 광성로6길 66,126.940168393772,37.547265277456,서울특별시 마포구 광성로6길 66
1,서울 마포구 대흥로 40,126.938931078126,37.5442301695085,서울특별시 마포구 대흥로 40
2,서울 마포구 마포대로1길 26,126.942323015666,37.540313878466,서울특별시 마포구 마포대로1길 26
3,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46
4,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46 우측상가


In [50]:
mapo_m = pd.merge(mapo_latlng, mapo_cbb, on=['도로명주소'], how='left')
mapo_m.drop_duplicates()
print(mapo_m.shape)
mapo_m.head()

(65, 10)


,address_name,경도,위도,도로명주소,시도명,시군구명,상호명,수량,세부위치,데이터기준일자
0,서울 마포구 광성로6길 66,126.940168393772,37.547265277456,서울특별시 마포구 광성로6길 66,서울특별시,마포구,마포나루장터,1,NaN,2022-08-05
1,서울 마포구 대흥로 40,126.938931078126,37.5442301695085,서울특별시 마포구 대흥로 40,서울특별시,마포구,블랙시스(아이나비),1,가게앞,2022-08-05
2,서울 마포구 마포대로1길 26,126.942323015666,37.540313878466,서울특별시 마포구 마포대로1길 26,서울특별시,마포구,성우서서갈비,2,가게안/가게앞,2022-08-05
3,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46,서울특별시,마포구,고고네 닭한마리,1,가게앞,2022-08-05
4,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46 우측상가,서울특별시,마포구,원마산아구찜,1,재활용수집,2022-08-05


In [51]:
mapo_m = mapo_m.drop(columns=['시도명', '데이터기준일자'])

In [52]:
mapo_m = mapo_m[['시군구명', 'address_name', '경도', '위도', '도로명주소', '상호명', '수량', '세부위치']]
mapo_m.head()

,시군구명,address_name,경도,위도,도로명주소,상호명,수량,세부위치
0,마포구,서울 마포구 광성로6길 66,126.940168393772,37.547265277456,서울특별시 마포구 광성로6길 66,마포나루장터,1,NaN
1,마포구,서울 마포구 대흥로 40,126.938931078126,37.5442301695085,서울특별시 마포구 대흥로 40,블랙시스(아이나비),1,가게앞
2,마포구,서울 마포구 마포대로1길 26,126.942323015666,37.540313878466,서울특별시 마포구 마포대로1길 26,성우서서갈비,2,가게안/가게앞
3,마포구,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46,고고네 닭한마리,1,가게앞
4,마포구,서울 마포구 마포대로1길 46,126.940477382241,37.5416404055047,서울특별시 마포구 마포대로1길 46 우측상가,원마산아구찜,1,재활용수집


In [53]:
mapo_m.to_csv('data/쓰레기통_위경도/마포구_담배꽁초쓰레기통_위경도.csv', index=False)

## 서초구 쓰레기통

In [54]:
seocho_cbb = pd.read_csv('data/쓰레기통/서울특별시 서초구_담배꽁초수거함 현황_20220816.csv', encoding='cp949')
seocho_cbb.head()

,자치구명,해당동,설치위치,설치주체,청소관리,설치일자,위도,경도,비고
0,서초구,서초4동,교보타워(강남대로 465) 부지주차장 진입로 옆,교보타워,교보타워,2012-04-26,37.503756,127.024110,NaN
1,서초구,양재1동,엘타워 뒤 주차장 진입로 옆,엘타워,엘타워,2015-04-18,37.482767,127.034950,NaN
2,서초구,양재1동,서울가정행정법원 청사 출입계단 밑,서울가정법원,서울가정법원,2015-06-08,37.481245,127.036190,NaN
3,서초구,반포4동,고속버스터미널역 3번출구 앞,센트럴시티,센트럴시티,2018-03-16,37.504492,127.007845,NaN
4,서초구,서초2동,강남역 7번 출구와 8번출구 사이 보도 서초동 1319-5 앞,서초구보건소,청소행정과,2018-08-28,37.497326,127.027281,NaN


In [55]:
seocho_cbb = seocho_cbb.rename(columns={'자치구명':'시군구명', '해당동':'동명', '설치위치':'설치주소'})

In [56]:
seocho_cbb[['시군구명', '동명', '경도', '위도', '설치주소', '설치주체', '청소관리', '설치일자', '비고']]

,시군구명,동명,경도,위도,설치주소,설치주체,청소관리,설치일자,비고
0,서초구,서초4동,127.024110,37.503756,교보타워(강남대로 465) 부지주차장 진입로 옆,교보타워,교보타워,2012-04-26,NaN
1,서초구,양재1동,127.034950,37.482767,엘타워 뒤 주차장 진입로 옆,엘타워,엘타워,2015-04-18,NaN
2,서초구,양재1동,127.036190,37.481245,서울가정행정법원 청사 출입계단 밑,서울가정법원,서울가정법원,2015-06-08,NaN
3,서초구,반포4동,127.007845,37.504492,고속버스터미널역 3번출구 앞,센트럴시티,센트럴시티,2018-03-16,NaN
4,서초구,서초2동,127.027281,37.497326,강남역 7번 출구와 8번출구 사이 보도 서초동 1319-5 앞,서초구보건소,청소행정과,2018-08-28,NaN
...,...,...,...,...,...,...,...,...,...
57,서초구,양재2동,127.041653,37.467537,양재동 351-1 도토리편백집앞,청소행정과,청소행정과,2019-10-22,NaN
58,서초구,양재1동,127.015931,37.457240,중앙로 584 에비뉴클래식 (우면동 760),청소행정과,청소행정과,2019-10-22,NaN
59,서초구,양재2동,127.042944,37.467059,양재동 349-27 블루핸즈 앞,청소행정과,청소행정과,2019-10-22,NaN
60,서초구,서초2동,127.031360,37.489195,서초동 1340-6 남강빌딩 흡연지역,청소행정과,청소행정과,2020-05-13,NaN


In [57]:
seocho_cbb.to_csv('data/쓰레기통_위경도/서초구_담배꽁초쓰레기통_위경도.csv', index=False)

## 양천구 쓰레기통

In [58]:
yangcheon_cbb = pd.read_csv('data/쓰레기통/서울특별시 양천구_담배꽁초쓰레기통_20220816.csv', encoding='cp949')
yangcheon_cbb.head()

,연번,설치장소,도로명주소,설치일자,데이터기준일자
0,1,현대41타워앞 인도,서울시 양천구 목동동로 293,2022-05-01,2022-08-16
1,2,파리지앙A앞 공용주차장,서울시 양천구 목동서로 57,2022-05-01,2022-08-16
2,3,보성상가옆 먹자골목,서울시 양천구 오목로 232,2022-05-01,2022-08-16
3,4,신정네거리역 뒤 먹자골목,서울시 양천구 중앙로 276,2022-05-01,2022-08-16
4,5,농협 옆 골목,서울시 양천구 오목로 325,2022-05-01,2022-08-16


In [59]:
yangcheon_cbb = yangcheon_cbb.drop(columns=['연번', '설치일자', '데이터기준일자'])

In [60]:
yangcheonlist = []
for i in list(yangcheon_cbb['도로명주소']):
    try:
        yangcheonlist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류: {e}')
        yangcheonlist.append(None)

In [61]:
[i for i in yangcheonlist if i==None]

[]

In [62]:
yangcheon_latlng = pd.DataFrame(yangcheonlist)
yangcheon_latlng.columns = ['address_name', '경도', '위도', '도로명주소']
yangcheon_latlng.head()

,address_name,경도,위도,도로명주소
0,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293
1,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57
2,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232
3,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276
4,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325


In [63]:
yangcheon_m = pd.merge(yangcheon_latlng, yangcheon_cbb, on=['도로명주소'], how='left')
yangcheon_m.drop_duplicates
print(yangcheon_m.shape)
yangcheon_m.head()

(12, 5)


,address_name,경도,위도,도로명주소,설치장소
0,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293,현대41타워앞 인도
1,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57,파리지앙A앞 공용주차장
2,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232,보성상가옆 먹자골목
3,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276,신정네거리역 뒤 먹자골목
4,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325,농협 옆 골목


In [64]:
yangcheon_m['시군구명'] = '양천구'

In [65]:
yangcheon_m[['시군구명', 'address_name', '경도', '위도', '도로명주소', '설치장소']]

,시군구명,address_name,경도,위도,도로명주소,설치장소
0,양천구,서울 양천구 목동동로 293,126.875771989415,37.5281783139269,서울시 양천구 목동동로 293,현대41타워앞 인도
1,양천구,서울 양천구 목동서로 57,126.879863427781,37.5366321308561,서울시 양천구 목동서로 57,파리지앙A앞 공용주차장
2,양천구,서울 양천구 오목로 232,126.863206293208,37.5257642505039,서울시 양천구 오목로 232,보성상가옆 먹자골목
3,양천구,서울 양천구 중앙로 276,126.85288429171,37.5208361584987,서울시 양천구 중앙로 276,신정네거리역 뒤 먹자골목
4,양천구,서울 양천구 오목로 325,126.873626981623,37.5251074073334,서울시 양천구 오목로 325,농협 옆 골목
5,양천구,서울 양천구 오목로 329,126.874124388629,37.5249139528398,서울시 양천구 오목로 329,오목교역 2번출구 옆
6,양천구,서울 양천구 목동동로 71,126.862972876451,37.5155139074803,서울시 양천구 목동동로 71,맥도날드 공용주차장
7,양천구,서울 양천구 목동로25길 2,126.863545739883,37.5279664201806,서울시 양천구 목동로25길 2,마스터그레이드 커피 앞
8,양천구,서울 양천구 신정중앙로 103,126.863371926417,37.5265013713269,서울시 양천구 신정중앙로 103,버스 안다니는 길 먹자골목
9,양천구,서울 양천구 오목로 153,126.854492996755,37.5250240257691,서울시 양천구 오목로 153,신정역 1번출구 GS 편의점 옆 골목


In [66]:
yangcheon_m.to_csv('data/쓰레기통_위경도/양천구_담배꽁초쓰레기통_위경도.csv', index=False)

## 영등포구 쓰레기통

In [67]:
yeongdeung_cbb = pd.read_csv('data/쓰레기통/서울특별시 영등포구_담배꽁초전용쓰레기통설치현황_20220803.csv', encoding='cp949')
yeongdeung_cbb.head()

,연번,동명,주소,비고,데이터기준일
0,1,당산1동,서울특별시 영등포구 당산로32길 1-6,NaN,2022-08-03
1,2,당산1동,서울특별시 영등포구 국회대로36길 7-3,NaN,2022-08-03
2,3,당산1동,서울특별시 영등포구 당산로32길 5 이차돌,NaN,2022-08-03
3,4,당산1동,서울특별시 영등포구 국회대로34길 4 가화,NaN,2022-08-03
4,5,당산1동,서울특별시 영등포구 국회대로34길 3,NaN,2022-08-03


In [68]:
yeongdeung_cbb = yeongdeung_cbb.drop(columns=['연번', '데이터기준일'])

In [69]:
yeongdeung_cbb.shape

(241, 3)

In [70]:
# 쓰레기통 주소만 확인하기 위해 개수는 고려하지 않고 중복 제거
yeongdeung_cbb.drop_duplicates(subset='주소', keep='first', inplace=True, ignore_index=True)

In [71]:
yeongdeung_cbb.shape

(206, 3)

In [75]:
yeongdeunglist = []
for i in list(yeongdeung_cbb['주소']):
    try:
        yeongdeunglist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류 {i}: {e}')
        yeongdeunglist.append(None)

In [73]:
# 주소 수정1
# 서울특별시 영등포구 당산로36길 14는 없고, 13과 15가 있음
# 서울특별시 영등포구 당산로38길 14가 있음. 일단 38길 14로 수정...
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 당산로36길 14', '주소'] = '서울특별시 영등포구 당산로38길 14'

# 주소 수정2
# 49길 2, 49길 4 있음. 49길 3 없음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 당산로49길3-3', '주소'] = '서울특별시 영등포구 당산로49길 2'

# 주소 수정3
# 서울특별시 영등포구 대림로8가길 2 가 있음
# 서울특별시 영등포구 대림로8길 3이 있음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대림로8길 2', '주소'] = '서울특별시 영등포구 대림로8가길 2'

# 주소 수정4
# 서울특별시 영등포구 디지털로 344 또는 346 있음
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 디지털로 345', '주소'] = '서울특별시 영등포구 디지털로 344'

# 주소 수정5
# 서울특별시 영등포구 도림로40길 의 가장 큰 주소 번호는 25. 도림로 40길로 변경
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림로40길 29', '주소'] = '서울특별시 영등포구 도림로40길'

# 주소 수정6 서울특별시 영등포구 문래 로데오 거리 주변
# 로데오거리가 위치한 문래동3가로 변경
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 문래 로데오 거리 주변', '주소'] = '서울특별시 영등포구 문래동3가'

# 주소 수정7 서울특별시 영등포구 RUST 빵집(경인로79길 15)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 RUST 빵집(경인로79길 15)', '주소'] = '서울특별시 영등포구 경인로79길 15'

# 주소 수정8 서울특별시 영등포구 퇴근길호프(도림로133길14-1)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 퇴근길호프(도림로133길14-1)', '주소'] = '서울특별시 영등포구 도림로133길14-1'

# 주소 수정9 서울특별시 영등포구 골목집(도림로139가길11)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 골목집(도림로139가길11)', '주소'] = '서울특별시 영등포구 도림로139가길11'

# 주소 수정10 서울특별시 영등포구 도림 로 98길 308 (성락교회 맞은편 과일가게와 공사현장 사이골목)
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림 로 98길 308 (성락교회 맞은편 과일가게와 공사현장 사이골목)', '주소'] = '서울특별시 영등포구 도림로 308'

# 주소 수정11 서울특별시 영등포구 도림로63길 20
# 아파트 단지로 변함. 제거가 나아보임
yeongdeung_cbb.drop(yeongdeung_cbb[yeongdeung_cbb['주소']=='서울특별시 영등포구 도림로63길 20'].index, inplace=True)


# 주소 수정12 서울특별시 영등포구 신풍로 13 3×5기사식당
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 신풍로 13 3×5기사식당', '주소'] = '서울특별시 영등포구 신풍로 13'

# 주소 수정13 서울특별시 영등포구 대방천로 191-1
# 근사값 191
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대방천로 191-1', '주소'] = '서울특별시 영등포구 대방천로 191'

# 주소 수정14 서울특별시 영등포구 대방천로 169
# 근사값 168
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 대방천로 169', '주소'] = '서울특별시 영등포구 대방천로 168'

# 주소 수정15 서울특별시 영등포구 선유로25길 태양 빌딩
# 근사값 선유로 25길 7
# 태양 빌딩 ' '빈 띄어쓰기 존재
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 선유로25길 태양 빌딩 ', '주소'] = '서울특별시 영등포구 선유로25길 7'

# 주소 수정16 서울특별시 영등포구 문래로 205
# 근사값 203
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 문래로 205', '주소'] = '서울특별시 영등포구 문래로 203'

# 주소 수정17 서울특별시 영등포구 영등포로33길 16(스위트빌) 1
# 스위트빌 주소는 33길 14
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 영등포로33길 16(스위트빌) 1', '주소'] = '서울특별시 영등포구 영등포로33길 14'

# 주소 수정18 서울특별시 영등포구 영등포로33길 16(스위트빌) 2
# 중복 제거
yeongdeung_cbb.drop(yeongdeung_cbb[yeongdeung_cbb['주소']=='서울특별시 영등포구 영등포로33길 16(스위트빌) 2'].index, inplace=True)

# 주소 수정19 서울특별시 영등포구 경인로112길 4
# 근사값 114가길 4
yeongdeung_cbb.loc[yeongdeung_cbb['주소']=='서울특별시 영등포구 경인로112길 4', '주소'] = '서울특별시 영등포구 경인로114가길 4'

In [76]:
yeongdeung_latlng = pd.DataFrame(yeongdeunglist)
yeongdeung_latlng.columns = ['address_name', '경도', '위도', '주소']
yeongdeung_latlng.head()

,address_name,경도,위도,주소
0,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6
1,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3
2,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌
3,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화
4,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3


In [77]:
yeongdeung_m = pd.merge(yeongdeung_latlng, yeongdeung_cbb, on=['주소'], how='left')
yeongdeung_m.drop_duplicates
print(yeongdeung_m.shape)
yeongdeung_m.head()

(204, 6)


,address_name,경도,위도,주소,동명,비고
0,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6,당산1동,NaN
1,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3,당산1동,NaN
2,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌,당산1동,NaN
3,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화,당산1동,NaN
4,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3,당산1동,NaN


In [78]:
yeongdeung_m['시군구명']='영등포구'

In [79]:
yeongdeung_m[['시군구명', '동명', 'address_name', '경도', '위도', '주소', '비고']]

,시군구명,동명,address_name,경도,위도,주소,비고
0,영등포구,당산1동,서울 영등포구 당산로32길 1-6,126.897266795561,37.5267406065353,서울특별시 영등포구 당산로32길 1-6,NaN
1,영등포구,당산1동,서울 영등포구 국회대로36길 7-3,126.898123773058,37.5265607881376,서울특별시 영등포구 국회대로36길 7-3,NaN
2,영등포구,당산1동,서울 영등포구 당산로32길 5,126.897473959605,37.5264757118622,서울특별시 영등포구 당산로32길 5 이차돌,NaN
3,영등포구,당산1동,서울 영등포구 국회대로34길 4,126.897451796745,37.5272118109642,서울특별시 영등포구 국회대로34길 4 가화,NaN
4,영등포구,당산1동,서울 영등포구 국회대로34길 3,126.897657998692,37.527154776227,서울특별시 영등포구 국회대로34길 3,NaN
...,...,...,...,...,...,...,...
199,영등포구,영등포동,서울 영등포구 경인로114가길 4,126.914000440734,37.5181395828305,서울특별시 영등포구 경인로114가길 4,NaN
200,영등포구,영등포본동,서울 영등포구 영신로15길 7,126.907613549018,37.5137102024436,서울특별시 영등포구 영신로15길 7 현대기술학원 정문,NaN
201,영등포구,영등포본동,서울 영등포구 영신로15길 7,126.907613549018,37.5137102024436,서울특별시 영등포구 영신로15길 7 현대기술학원 후문,NaN
202,영등포구,영등포본동,서울 영등포구 영신로17길,126.907247554301,37.5138686724654,서울특별시 영등포구 영신로17길,NaN


In [80]:
yeongdeung_m.to_csv('data/쓰레기통_위경도/영등포구_담배꽁초쓰레기통_위경도.csv', index=False)

## 중랑구 쓰레기통

In [81]:
jungnang_cbb = pd.read_csv('data/쓰레기통/서울특별시 중랑구_담배꽁초쓰레기통_20220804.csv', encoding='cp949')
jungnang_cbb

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,묵동 122-361 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,묵1동 주민센터 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,중랑역로 100 (디엠씨빌딩 앞),담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,동일로 101길 20-15 주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,6,겸재로,겸재로3길 50 서부경로당,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,7,겸재로,겸재로122 박가네추어탕 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [82]:
jungnang_cbb['설치장소'] = jungnang_cbb['설치장소'].apply(lambda x: x.split('(')[0])
jungnang_cbb.head()

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,묵동 122-361 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,묵1동 주민센터 앞,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,중랑역로 100,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,동일로 101길 20-15 주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [83]:
a = jungnang_cbb['설치장소'].str.split(' ')
a

0                [상봉동, 130-3]
1            [묵동, 122-361, 앞]
2              [묵1동, 주민센터, 앞]
3               [중랑역로, 100, ]
4    [동일로, 101길, 20-15, 주민센터]
5          [겸재로3길, 50, 서부경로당]
6         [겸재로122, 박가네추어탕, 앞]
Name: 설치장소, dtype: object

In [84]:
for i in range(len(a)):
    if a[i][-1] == '맞은편':
        a[i] = a[i][:-1]
    elif a[i][-1] == '앞':
        a[i] = a[i][:-1]

In [85]:
for i in range(len(a)):
    if len(a[i]) == 1:
        a[i] = '서울특별시 중랑구 ' + a[i][0]
    else:
        a[i] = '서울특별시 중랑구 ' + a[i][0] + ' ' + a[i][1]

In [86]:
a

0        서울특별시 중랑구 상봉동 130-3
1       서울특별시 중랑구 묵동 122-361
2         서울특별시 중랑구 묵1동 주민센터
3         서울특별시 중랑구 중랑역로 100
4         서울특별시 중랑구 동일로 101길
5         서울특별시 중랑구 겸재로3길 50
6    서울특별시 중랑구 겸재로122 박가네추어탕
Name: 설치장소, dtype: object

In [87]:
jungnang_cbb['설치장소'] = a

In [88]:
# 오류로 주소 변경
jungnang_cbb.loc[jungnang_cbb['설치장소']=='서울특별시 중랑구 묵1동 주민센터', '설치장소'] = '서울특별시 중랑구 공릉로2가길 5 묵1동주민센터'
jungnang_cbb

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,망우로,서울특별시 중랑구 상봉동 130-3,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,2,동일로,서울특별시 중랑구 묵동 122-361,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,3,공릉로,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,4,중랑역로,서울특별시 중랑구 중랑역로 100,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,5,동일로,서울특별시 중랑구 동일로 101길,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,6,겸재로,서울특별시 중랑구 겸재로3길 50,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,7,겸재로,서울특별시 중랑구 겸재로122 박가네추어탕,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [89]:
jungnanglist = []
for i in list(jungnang_cbb['설치장소']):
    try:
        jungnanglist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류 {i}: {e}')
        jungnanglist.append(None)

In [90]:
jungnang_latlng = pd.DataFrame(jungnanglist)
jungnang_latlng.columns = ['address_name', '경도', '위도', '설치장소']
jungnang_latlng.head()

,address_name,경도,위도,설치장소
0,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3
1,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361
2,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터
3,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100
4,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길


In [91]:
jungnang_m = pd.merge(jungnang_latlng, jungnang_cbb, on=['설치장소'], how='left')
jungnang_m.drop_duplicates
print(jungnang_m.shape)
jungnang_m.head()

(7, 9)


,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태
0,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3,1,망우로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361,2,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,3,공릉로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100,4,중랑역로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길,5,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [92]:
jungnang_m['시군구명']='중랑구'

In [93]:
jungnang_m[['시군구명', 'address_name', '경도', '위도', '설치장소', '연번', '도로명', '설치 장소 유형',
       '수거 쓰레기 종류', '형태']]

,시군구명,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태
0,중랑구,서울 중랑구 상봉동 130-3,127.074681320491,37.5932328015902,서울특별시 중랑구 상봉동 130-3,1,망우로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
1,중랑구,서울 중랑구 묵동 122-361,127.078312444399,37.6092671767219,서울특별시 중랑구 묵동 122-361,2,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
2,중랑구,서울 중랑구 공릉로2가길 5,127.078479291591,37.612414580688,서울특별시 중랑구 공릉로2가길 5 묵1동주민센터,3,공릉로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
3,중랑구,서울 중랑구 중랑역로 100,127.077797714827,37.6024297259926,서울특별시 중랑구 중랑역로 100,4,중랑역로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
4,중랑구,서울 중랑구 동일로101길,127.076756397416,37.5888366116663,서울특별시 중랑구 동일로 101길,5,동일로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
5,중랑구,서울 중랑구 겸재로3길 50,127.076907543484,37.5881687878315,서울특별시 중랑구 겸재로3길 50,6,겸재로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함
6,중랑구,서울 중랑구 겸재로 122,127.081347905764,37.5868055827869,서울특별시 중랑구 겸재로122 박가네추어탕,7,겸재로,담배꽁초 무단투기 취약지역,담배꽁초,담배꽁초 수거함


In [94]:
jungnang_m.to_csv('data/쓰레기통_위경도/중랑구_담배꽁초쓰레기통_위경도.csv', index=False)

## 종로구 쓰레기통

In [95]:
jongno_cbb = pd.read_csv('data/쓰레기통/서울특별시 종로구_담배꽁초쓰레기통_20220713.csv', encoding='cp949')
jongno_cbb.head()

,연번,도로명,설치장소,설치 장소 유형,수거 쓰레기 종류,형태
0,1,자하문로,자하문로1길 12,상가지역,담배꽁초,담배꽁초 수거함
1,2,자하문로,자하문로1길 50-1,상가지역,담배꽁초,담배꽁초 수거함
2,3,자하문로,자하문로1길 34,상가지역,담배꽁초,담배꽁초 수거함
3,4,자하문로,자하문로1길 49-4,상가지역,담배꽁초,담배꽁초 수거함
4,5,자하문로,자하문로 103-19,상가지역,담배꽁초,담배꽁초 수거함


In [96]:
address = '자하문로1길 12'
url = "https://dapi.kakao.com/v2/local/search/address.json?query={}".format(address)
headers = {"Authorization": 'KakaoAK 9b47700a217b794a8878d97b26b723be'}
results = requests.get(url, headers=headers).json()['documents']
results

[{'address': {'address_name': '서울 종로구 체부동 57',
   'b_code': '1111011200',
   'h_code': '1111053000',
   'main_address_no': '57',
   'mountain_yn': 'N',
   'region_1depth_name': '서울',
   'region_2depth_name': '종로구',
   'region_3depth_h_name': '사직동',
   'region_3depth_name': '체부동',
   'sub_address_no': '',
   'x': '126.97174802679',
   'y': '37.576573141464'},
  'address_name': '서울 종로구 자하문로1길 12',
  'address_type': 'ROAD_ADDR',
  'road_address': {'address_name': '서울 종로구 자하문로1길 12',
   'building_name': '',
   'main_building_no': '12',
   'region_1depth_name': '서울',
   'region_2depth_name': '종로구',
   'region_3depth_name': '체부동',
   'road_name': '자하문로1길',
   'sub_building_no': '',
   'underground_yn': 'N',
   'x': '126.97174802679',
   'y': '37.576573141464',
   'zone_no': '03041'},
  'x': '126.97174802679',
  'y': '37.576573141464'}]

In [100]:
jongnolist = []
for i in list(jongno_cbb['설치장소']):
    try:
        jongnolist.append(GetLatLng(i))
    except Exception as e:
        print(f'주소 검색 오류 {i}: {e}')
        jongnolist.append(None)

In [98]:
# 주소 수정1 '자하문로 103-19'
# -19번지는 존재하지 않음.
# 하지만 자하문로 103은 학교 정문이라서 고민이 필요
jongno_cbb.loc[jongno_cbb['설치장소']=='자하문로 103-19', '설치장소'] = '자하문로 103'

# 주소 수정2 '자하문로1길 103-4'
# 자하문로1길과 크로스되는 곳에 사직로 103-4 존재
jongno_cbb.loc[jongno_cbb['설치장소']=='자하문로1길 103-4', '설치장소'] = '사직로 103-4'

# 주소 수정3 '자하문로1길 3-2'
# 자하문로 1길 3-1 또는 3 존재
jongno_cbb.loc[jongno_cbb['설치장소']=='자하문로1길 3-2', '설치장소'] = '자하문로1길 3-1'

In [101]:
jongno_latlng = pd.DataFrame(jongnolist)
jongno_latlng.columns = ['address_name', '경도', '위도', '설치장소']
jongno_latlng.head()

,address_name,경도,위도,설치장소
0,서울 종로구 자하문로1길 12,126.97174802679,37.576573141464,자하문로1길 12
1,서울 종로구 자하문로1길 50-1,126.96968004129,37.5770099710907,자하문로1길 50-1
2,서울 종로구 자하문로1길 34,126.970550412243,37.576612402426,자하문로1길 34
3,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4
4,서울 종로구 자하문로 103,126.969935904726,37.5851302421868,자하문로 103


In [102]:
jongno_m = pd.merge(jongno_latlng, jongno_cbb, on=['설치장소'], how='left')
jongno_m.drop_duplicates
print(jongno_m.shape)
jongno_m.head()

(38, 9)


,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태
0,서울 종로구 자하문로1길 12,126.97174802679,37.576573141464,자하문로1길 12,1,자하문로,상가지역,담배꽁초,담배꽁초 수거함
1,서울 종로구 자하문로1길 50-1,126.96968004129,37.5770099710907,자하문로1길 50-1,2,자하문로,상가지역,담배꽁초,담배꽁초 수거함
2,서울 종로구 자하문로1길 34,126.970550412243,37.576612402426,자하문로1길 34,3,자하문로,상가지역,담배꽁초,담배꽁초 수거함
3,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,4,자하문로,상가지역,담배꽁초,담배꽁초 수거함
4,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,8,자하문로,상가지역,담배꽁초,담배꽁초 수거함


In [103]:
jongno_m['시군구명']='종로구'

In [104]:
jongno_m[['시군구명', 'address_name', '경도', '위도', '설치장소', '연번', '도로명', '설치 장소 유형',
       '수거 쓰레기 종류', '형태']]
jongno_m.head()

,address_name,경도,위도,설치장소,연번,도로명,설치 장소 유형,수거 쓰레기 종류,형태,시군구명
0,서울 종로구 자하문로1길 12,126.97174802679,37.576573141464,자하문로1길 12,1,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
1,서울 종로구 자하문로1길 50-1,126.96968004129,37.5770099710907,자하문로1길 50-1,2,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
2,서울 종로구 자하문로1길 34,126.970550412243,37.576612402426,자하문로1길 34,3,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
3,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
4,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,8,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구


In [105]:
jongno_m.drop(columns={'연번'})

,address_name,경도,위도,설치장소,도로명,설치 장소 유형,수거 쓰레기 종류,형태,시군구명
0,서울 종로구 자하문로1길 12,126.97174802679,37.576573141464,자하문로1길 12,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
1,서울 종로구 자하문로1길 50-1,126.96968004129,37.5770099710907,자하문로1길 50-1,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
2,서울 종로구 자하문로1길 34,126.970550412243,37.576612402426,자하문로1길 34,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
3,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
4,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
5,서울 종로구 자하문로 103,126.969935904726,37.5851302421868,자하문로 103,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
6,서울 종로구 자하문로1길 33,126.970393335988,37.5764790159668,자하문로1길 33,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
7,서울 종로구 사직로 103-4,126.970607440318,37.5763941957968,사직로 103-4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
8,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구
9,서울 종로구 자하문로1길 49-4,126.969547804414,37.5767621636113,자하문로1길 49-4,자하문로,상가지역,담배꽁초,담배꽁초 수거함,종로구


In [106]:
jongno_m.to_csv('data/쓰레기통_위경도/종로구_담배꽁초쓰레기통_위경도.csv', index=False)